## Populate Task

In [1]:
PATH_FILE_TASK = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/tasks/task/'
PATH_FILE_RESULT = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/tasks/result/'
PATH_FILE_TEMPLATE = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/tasks/task_template.json'
PATH_FILE_DATA = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/data/mave_filtered_test.jsonl'

In [2]:
task = 'update_open_one_shot'

In [3]:
from dotenv import load_dotenv
import json
from pieutils import update_task_dict_from_file, combine_example, calculate_recall_precision_f1, parse_gpt_response_to_dict

# Load OPENAI_API_KEY from .env file
load_dotenv()

True

In [4]:
category_attribute_2_human_msg = {
    'Flash Memory Cards_SD Format': ['16GB MicroSDHC Memory Card for AT&T LG Xpression C395 Smartphone with Free USB MicroSD/SDHC Card Reader -- 16 G/GB/GIG 16G 16GIG',],
    'Flash Memory Cards_Capacity': ['16GB MicroSDHC Memory Card for AT&T LG Xpression C395 Smartphone with Free USB MicroSD/SDHC Card Reader -- 16 G/GB/GIG 16G 16GIG',],
    'Digital Cameras_Optical Zoom': ['Pentax Optio 230 2MP Digital Camera w/ 3x Optical Zoom',],
    'Digital Cameras_Camera Weight': ['Pentax Optio 230 2MP Digital Camera w/ 3x Optical Zoom',],
    'Digital Cameras_Resolution': ['Pentax Optio 230 2MP Digital Camera w/ 3x Optical Zoom',],
    'Digital Cameras_Sensor Size': ['Sony a7 Full-Frame Mirrorless Digital Camera with 28-70mm Lens',],
    'Digital Cameras_Sensor Type': ['Nikon D3300 24.2 MP CMOS Digital SLR Body Only (Black) Bundle 1',],
    'Laptops_Processor Brand': ['Acer Aspire S7 13.3-Inch Touchscreen Ultrabook (Intel Core i5-4200U Processor up to 2.6GHz, 8GB DDR3 Memory, 256GB SSD, FHD 1080P Touchscreen Display, Backlit Keyboard, 802.11AC WiFi, Windows 8.1, 0.5 inch thin and 2.9 Lbs) Crystal White', ],
    'Laptops_Resolution': ['Acer Aspire S7 13.3-Inch Touchscreen Ultrabook (Intel Core i5-4200U Processor up to 2.6GHz, 8GB DDR3 Memory, 256GB SSD, FHD 1080P Touchscreen Display, Backlit Keyboard, 802.11AC WiFi, Windows 8.1, 0.5 inch thin and 2.9 Lbs) Crystal White',],
    'Laptops_Battery Life': ['Gateway EC1430U 11.6-Inch Black Laptop - Up to 7 Hours of Battery Life (Windows 7 Home Premium)', 'Acer Aspire One AOD250-1289 10.1-Inch Seashell White Netbook - 6.5 Hour Battery Life', 'ThinkPad T22 2647 - PIII 900 MHz - RAM 128 MB - HDD 20 GB - DVD - Savage/IX - Win2000 Pro - 14.1\" TFT 1024 x 768 ( XGA ) - black'],
    'Laptops_Number of Cores': ['HP Pavilion dv6113us 15.4\" Notebook Laptop PC (AMD Turion 64 X2 Dual-Core TL-50 1.6GHz, 1GB RAM, 80GB HDD / DVD\u00b1RW DL / Wireless / WXP Media Center) by CE',],
    'Laptops_Processor Speed': ['Acer Aspire S7 13.3-Inch Touchscreen Ultrabook (Intel Core i5-4200U Processor up to 2.6GHz, 8GB DDR3 Memory, 256GB SSD, FHD 1080P Touchscreen Display, Backlit Keyboard, 802.11AC WiFi, Windows 8.1, 0.5 inch thin and 2.9 Lbs) Crystal White', ],
    'Laptops_Screen Size': ['Acer Aspire S7 13.3-Inch Touchscreen Ultrabook (Intel Core i5-4200U Processor up to 2.6GHz, 8GB DDR3 Memory, 256GB SSD, FHD 1080P Touchscreen Display, Backlit Keyboard, 802.11AC WiFi, Windows 8.1, 0.5 inch thin and 2.9 Lbs) Crystal White', 'ThinkPad T22 2647 - PIII 900 MHz - RAM 128 MB - HDD 20 GB - DVD - Savage/IX - Win2000 Pro - 14.1\" TFT 1024 x 768 ( XGA ) - black', 'Compaq Presario 2700US Laptop (1-GHz Pentium III, 256 MB RAM, 20 GB hard drive)'],
    'Laptops_Weight': ['Acer Aspire S7 13.3-Inch Touchscreen Ultrabook (Intel Core i5-4200U Processor up to 2.6GHz, 8GB DDR3 Memory, 256GB SSD, FHD 1080P Touchscreen Display, Backlit Keyboard, 802.11AC WiFi, Windows 8.1, 0.5 inch thin and 2.9 Lbs) Crystal White', 'ThinkPad T22 2647 - PIII 900 MHz - RAM 128 MB - HDD 20 GB - DVD - Savage/IX - Win2000 Pro - 14.1\" TFT 1024 x 768 ( XGA ) - black', 'Apple iBook Laptop (500-MHz PowerPC G3, 64 MB RAM, 10 GB hard drive)'],
    'Laptops_Refresh Rate': ['Acer Aspire S7 13.3-Inch Touchscreen Ultrabook (Intel Core i5-4200U Processor up to 2.6GHz, 8GB DDR3 Memory, 256GB SSD, FHD 1080P Touchscreen Display, Backlit Keyboard, 802.11AC WiFi, Windows 8.1, 0.5 inch thin and 2.9 Lbs) Crystal White', '2018 Newest Flagship Asus ROG Strix 17.3 Full HD 120Hz VR Ready Gaming Laptop (i5-2430M | 8GB RAM | 640GB HDD | Windows 7 Home)', 'ThinkPad T22 2647 - PIII 900 MHz - RAM 128 MB - HDD 20 GB - DVD - Savage/IX - Win2000 Pro - 14.1\" TFT 1024 x 768 ( XGA ) - black']
}

category_attribute_2_system_msg = {
    'Flash Memory Cards_SD Format': ['Capacity: 16GB \n SD Format: MicroSDHC'],
    'Flash Memory Cards_Capacity': ['Capacity: 16GB \n SD Format: MicroSDHC'],
    'Digital Cameras_Optical Zoom': ['Resolution: 2MP \n Optical Zoom: 3x \n Camera Weight: I do not know. \n Sensor Size: I do not know. \n Sensor Type: I do not know.'],
    'Digital Cameras_Camera Weight': ['Resolution: 2MP \n Optical Zoom: 3x \n Camera Weight: I do not know. \n Sensor Size: I do not know. \n Sensor Type: I do not know.'],
    'Digital Cameras_Resolution': ['Resolution: 2MP \n Optical Zoom: 3x \n Camera Weight: I do not know. \n Sensor Size: I do not know. \n Sensor Type: I do not know.'],
    'Digital Cameras_Sensor Size': ['Resolution: I do not know. \n Optical Zoom: I do not know. \n Camera Weight: I do not know. \n Sensor Size: Full-Frame \n Sensor Type: I do not know.'],
    'Digital Cameras_Sensor Type': ['Resolution: I do not know. \n Optical Zoom: I do not know. \n Camera Weight: I do not know. \n Sensor Size: I do not know. \n Sensor Type: CMOS'],
    'Laptops_Processor Brand': ['Battery Life: I do not know. \n Number of Cores: I do not know. \n Processor Brand: Intel \n Processor Speed: 2.6GHz \n Refresh Rate: I do not know. \n Resolution: FHD 1080P \n Screen Size: 13.3 inches \n  Weight: 2.9 lbs'],
    'Laptops_Resolution': ['Battery Life: I do not know. \n Number of Cores: I do not know. \n Processor Brand: Intel \n Processor Speed: 2.6GHz \n Refresh Rate: I do not know. \n Resolution: FHD 1080P \n Screen Size: 13.3 inches \n  Weight: 2.9 lbs'],
    'Laptops_Battery Life': ['Battery Life: 7 Hours \n Number of Cores: I do not know. \n Processor Brand: I do not know. \n Processor Speed: I do not know. \n Refresh Rate: I do not know. \n Resolution: I do not know. \n Screen size: 11.6 inches \n Weight: I do not know.'],
    'Laptops_Number of Cores': ['Battery Life: I do not know. \n Number of Cores: 2 \n Processor Brand: I do not know. \n Processor Speed: I do not know. \n Refresh Rate: I do not know. \n Resolution: I do not know. \n Screen Size: 15.4 inches \n Weight: I do not know.'],
    'Laptops_Processor Speed': ['Battery Life: I do not know. \n Number of Cores: I do not know. \n Processor Brand: Intel \n Processor Speed: 2.6GHz \n Refresh Rate: I do not know. \n Resolution: FHD 1080P \n Screen Size: 13.3 inches \n  Weight: 2.9 lbs'],
    'Laptops_Screen Size': ['Battery Life: I do not know. \n Number of Cores: I do not know. \n Processor Brand: Intel \n Processor Speed: 2.6GHz \n Refresh Rate: I do not know. \n Resolution: FHD 1080P \n Screen Size: 13.3 inches \n  Weight: 2.9 lbs'],
    'Laptops_Weight': ['Battery Life: I do not know. \n Number of Cores: I do not know. \n Processor Brand: Intel \n Processor Speed: 2.6GHz \n Refresh Rate: I do not know. \n Resolution: FHD 1080P \n Screen Size: 13.3 inches \n  Weight: 2.9 lbs'],
    'Laptops_Refresh Rate': ['Battery Life: I do not know. \n Number of Cores: I do not know. \n Processor Brand: Intel \n Processor Speed: 2.6GHz \n Refresh Rate: I do not know. \n Resolution: FHD 1080P \n Screen Size: 13.3 inches \n  Weight: 2.9 lbs']
}

In [5]:
# Update Task Dict
def update_task_dict_from_file(file_name, task_dict, flag):
    with open(file_name, 'r') as f:
        for line in f:
            record = json.loads(line)
            example = {}
            for attribute in record['attributes']:
                if attribute['key'] != record['attribute']:
                    continue
                
                example['human_0'] = category_attribute_2_human_msg['{}_{}'.format(record['category'], attribute['key'])][0]
                example['system_0'] = category_attribute_2_system_msg['{}_{}'.format(record['category'], attribute['key'])][0]
                example['input'] = record['title']
                example['category'] = record['category']
                if example['category'] == 'Flash Memory Cards':
                    example['attributes'] = '"Capacity", "SD Format"'
                elif example['category'] == 'Digital Cameras':
                    example['attributes'] = '"Resolution", "Optical Zoom", "Camera Weight", "Sensor Size", "Sensor Type"'
                elif example['category'] == 'Laptops':        
                    example['attributes'] = '"Battery Life", Number of Cores, Processor Brand, Processor Speed, Refresh Rate, Resolution, Screen Size, Weight'
                example['attribute'] = attribute['key']
                example['target_scores'] = {}

                for evidence in attribute['evidences']:
                    if evidence['pid'] == 0:
                        example['target_scores'][evidence['value']] = 1
                if len(example['target_scores']) == 0:
                    example['target_scores']['I do not know.'] = 1
                task_dict['examples'].append(example)
    return task_dict

In [6]:
# Load task template
with open(PATH_FILE_TEMPLATE, 'r') as f:
    task_dict = json.load(f)

task_dict['task_prefix'] = "Extract only attributes: {attributes} can mentioned in the product title below and return attribute values in the product title. If the attribute is not applicable or the attribute value cannot be derived from the product title, respond with 'I do not know.'. All parts of the product title should be assigned to an attribute. \n Desired format: <attribute name>: <attribute value> \n <attribute name>: <attribute value>"
task_dict['shots'] = 1

# Generate examples
task_dict = update_task_dict_from_file(PATH_FILE_DATA, task_dict, False)

with open(PATH_FILE_TASK + 'task_{}.json'.format(task), 'w', encoding='utf-8') as f:
    json.dump(task_dict, f, indent=4)

## Evaluate Task

In [7]:
# Initialize processing
from tqdm import tqdm
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

# Put task prefix into system message.
prompt_list = []
system_message_prompt = SystemMessagePromptTemplate.from_template(task_dict['task_prefix'])
prompt_list.append(system_message_prompt)

# Prepare 1-shot  
human_example_template="{human_msg_0}"
human_example_message_prompt = HumanMessagePromptTemplate.from_template(human_example_template)
prompt_list.append(human_example_message_prompt)

ai_example_template="{ai_msg_0}"
ai_example_message_prompt = AIMessagePromptTemplate.from_template(ai_example_template)
prompt_list.append(ai_example_message_prompt)

system_message_prompt_2 = SystemMessagePromptTemplate.from_template(task_dict['task_prefix'])
prompt_list.append(system_message_prompt_2)

# Add 
human_template="{input_string}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
prompt_list.append(human_message_prompt)

chat_prompt = ChatPromptTemplate.from_messages(prompt_list)

gpt_3_turbo = ChatOpenAI(model_name='gpt-3.5-turbo-0125', temperature=0)

llm_chain = LLMChain(
    prompt=chat_prompt,
    llm=gpt_3_turbo
)

# Save model
task_dict['model'] = llm_chain.llm.model_name

In [8]:
def select_and_run_llm(attributes, human_0, system_0, input):
        pred = None
        if len(input) == 0:
            # No input text provided.
            return pred
        
        response = llm_chain.run({'attributes': attributes, 'human_msg_0': human_0, 'ai_msg_0': system_0, 'input_string': input})
        try:
            print(response)
            pred = parse_gpt_response_to_dict(response)
        except Exception as e:
            print(e)
            print('Response: ')
            print(response)
        return pred

In [ ]:
# General task prefix
from tqdm import tqdm
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    preds = [select_and_run_llm(example['attributes'], example['human_0'], example['system_0'], example['input']) for example in tqdm(task_dict['examples'])]
    
    task_dict['total_tokens'] = cb.total_tokens
    print(f"Total Tokens: {cb.total_tokens}")

In [10]:
print(preds)

[{'Capacity': '64GB', 'SD Format': 'MicroSDHC'}, {'Capacity': '16GB', 'SD Format': 'MicroSDHC'}, {'Capacity': '32GB', 'SD Format': 'MicroSDHC'}, {'Capacity': '32GB', 'SD Format': 'MicroSDHC'}, {'Capacity': '4GB', 'SD Format': 'MicroSDHC'}, {'Capacity': '4GB', 'SD Format': 'MicroSDHC'}, {'Capacity': '32GB', 'SD Format': 'MicroSDHC'}, {'Capacity': '64GB', 'SD Format': 'MicroSDXC'}, {'Capacity': '64GB', 'SD Format': 'MicroSDXC'}, {'Capacity': '64GB', 'SD Format': 'MicroSDXC'}, {'Capacity': '4GB', 'SD Format': 'MicroSDHC'}, {'Capacity': '4GB', 'SD Format': 'MicroSDHC'}, {'Capacity': '4GB', 'SD Format': 'MicroSDHC'}, {'Capacity': '16 MB', 'SD Format': 'SmartMedia'}, {'Resolution': '2MP', 'Optical Zoom': '3x', 'Camera Weight': 'I do not know.', 'Sensor Size': 'I do not know.', 'Sensor Type': 'I do not know.'}, {'Capacity': '64MB', 'SD Format': 'I do not know.'}, {'Capacity': '64 MB', 'SD Format': 'I do not know.'}, {'Resolution': '2MP', 'Optical Zoom': '3x', 'Camera Weight': 'I do not know.'

In [11]:
# # Load task template
# with open(PATH_FILE_RESULT + 'task_run_chat_open_one_shot_gpt-3.5-turbo-0125.json', 'r') as f:
#     task_dict = json.load(f)

targets = [example['target_scores'] for example in task_dict['examples']]
categories = [example['category'] for example in task_dict['examples']]
attributes = [example['attribute'] for example in task_dict['examples']]

postprocessed_preds = [pred[attribute] if attribute in pred else "I do not know." for pred, attribute in zip(preds, attributes)]
#postprocessed_preds = [example['post_pred'] for example in task_dict['examples']]

task_dict['examples'] = [combine_example(example, pred, post_pred) 
                     for example, pred, post_pred in zip(task_dict['examples'], preds, postprocessed_preds)]

results = calculate_recall_precision_f1(targets, postprocessed_preds, categories, attributes)

Attribute: SD Format - Category: Flash Memory Cards
{'precision': 82.22, 'recall': 90.24, 'f1': 86.04}
Attribute: Capacity - Category: Flash Memory Cards
{'precision': 100.0, 'recall': 100.0, 'f1': 100.0}
Attribute: Resolution - Category: Laptops
{'precision': 87.8, 'recall': 87.8, 'f1': 87.8}
Attribute: Number of Cores - Category: Laptops
{'precision': 86.96, 'recall': 100.0, 'f1': 93.03}
Attribute: Screen Size - Category: Laptops
{'precision': 100.0, 'recall': 100.0, 'f1': 100.0}
Attribute: Battery Life - Category: Laptops
{'precision': 100.0, 'recall': 100.0, 'f1': 100.0}
Attribute: Processor Brand - Category: Laptops
{'precision': 100.0, 'recall': 97.73, 'f1': 98.85}
Attribute: Processor Speed - Category: Laptops
{'precision': 95.12, 'recall': 88.64, 'f1': 91.77}
Attribute: Resolution - Category: Digital Cameras
{'precision': 95.24, 'recall': 97.56, 'f1': 96.39}
Attribute: Sensor Size - Category: Digital Cameras
{'precision': 85.71, 'recall': 85.71, 'f1': 85.71}
Attribute: Optical 

In [12]:
# Save populated task
result_file = PATH_FILE_RESULT + 'task_run_chat_{}_{}_tenth.json'.format(task, task_dict['model'])
with open(result_file, 'w') as fp:
    json.dump(task_dict, fp, indent=4)

In [13]:
# Error Analysis
print('Prompts for which target and postprocessed prediction do not match.')
print('-----------')
input_texts = [example['input'] for example in task_dict['examples']]
preds = [example['pred'] for example in task_dict['examples']]

for input_text, target, pred, post_pred in zip(input_texts, targets, preds, postprocessed_preds):
    if post_pred not in target.keys():
        #print('Prompt: {}'.format(chat_prompt.format(task_prefix=task_dict['task_prefix'], input_string= input_text, human_msg_0= human_text_0, ai_msg=system_text,)))
        print('Input: {}'.format(input_text))
        print('Prediction: \n {}'.format(pred))
        print('Prediction 2: {}'.format(post_pred))
        print('Target: {}'.format(target))
        print('-----------')

Prompts for which target and postprocessed prediction do not match.
-----------
Input: HP Compaq Business Notebook nc4200 - Pentium M 750 / 1.86 GHz - Centrino - RAM 512 MB - HDD 60 GB - GMA 900 - Gigabit Ethernet - WLAN : 802.11b/g, Bluetooth - Win XP Pro - 12.1" TFT 1024 x 768 ( XGA )
Prediction: 
 {'Battery Life': 'I do not know.', 'Number of Cores': 'I do not know.', 'Processor Brand': 'Pentium M', 'Processor Speed': '1.86 GHz', 'Refresh Rate': 'I do not know.', 'Resolution': '1024 x 768 (XGA)', 'Screen Size': '12.1 inches', 'Weight': 'I do not know.'}
Prediction 2: 1024 x 768 (XGA)
Target: {'1024 x 768': 1, 'XGA': 1, '1024 x 768 ( XGA )': 1, '1024x768': 1}
-----------
Input: 16GB MicroSD Memory Card for HTC Wildfire Mobile/Cell Phone with Free USB Micro SD/SDHC Card Reader -- 16 G/GB/GIG 16G 16GIG MicroSDHC
Prediction: 
 {'Capacity': '16GB', 'SD Format': 'MicroSD'}
Prediction 2: MicroSD
Target: {'MicroSDHC': 1, 'microSDHC': 1}
-----------
Input: 8GB MicroSD Memory Card for Verizon